In [2]:
import zipfile
import xml.etree.ElementTree as ET
from odf.opendocument import load
from odf.table import Table, TableRow, TableCell
from odf.text import P
import os
import shutil

def read_guests_data(ods_file):
    doc = load(ods_file)
    table = doc.getElementsByType(Table)[0]
    data = []
    
    rows = table.getElementsByType(TableRow)
    header = None
    
    for i, row in enumerate(rows):
        cells = row.getElementsByType(TableCell)
        row_data = []
        for cell in cells:
            paragraphs = cell.getElementsByType(P)
            cell_text = ''.join(p.firstChild.data if p.firstChild else '' for p in paragraphs)
            row_data.append(cell_text)
        
        if i == 0:
            header = row_data
        else:
            if len(row_data) >= 2:
                data.append(dict(zip(header, row_data)))
    
    return data

def create_multi_page_odg(template_file, ods_file, output_file):
    guests_data = read_guests_data(ods_file)
    
    temp_dir = "temp_odg"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    
    with zipfile.ZipFile(template_file, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    
    content_file = os.path.join(temp_dir, "content.xml")
    tree = ET.parse(content_file)
    root = tree.getroot()
    
    namespaces = {
        'draw': 'urn:oasis:names:tc:opendocument:xmlns:drawing:1.0',
        'text': 'urn:oasis:names:tc:opendocument:xmlns:text:1.0', 
        'office': 'urn:oasis:names:tc:opendocument:xmlns:office:1.0'
    }
    
    body = root.find('.//office:body', namespaces)
    if body is None:
        raise ValueError("Не найден body в документе")
    
    office_drawing = body.find('.//office:drawing', namespaces)
    if office_drawing is None:
        raise ValueError("Не найден office:drawing в документе")
    
    pages = office_drawing.findall('draw:page', namespaces)
    if not pages:
        raise ValueError("Не найдены страницы в шаблоне")
    
    template_page = pages[0]
    page_index = list(office_drawing).index(template_page)

    for guest in guests_data:
        new_page = ET.fromstring(ET.tostring(template_page, encoding='unicode'))
        
        for elem in new_page.iter():
            if elem.text and '%Username%' in elem.text:
                elem.text = elem.text.replace('%Username%', guest.get('Username', ''))
            if elem.text and '%Row%' in elem.text:
                elem.text = elem.text.replace('%Row%', guest.get('Row', ''))
            
            if elem.tail and '%Username%' in elem.tail:
                elem.tail = elem.tail.replace('%Username%', guest.get('Username', ''))
            if elem.tail and '%Row%' in elem.tail:
                elem.tail = elem.tail.replace('%Row%', guest.get('Row', ''))
        
        office_drawing.insert(page_index, new_page)
        page_index += 1
    
    office_drawing.remove(template_page)
    
    tree.write(content_file, encoding='UTF-8', xml_declaration=True)
    
    with zipfile.ZipFile(output_file, 'w') as zipf:
        for root_dir, dirs, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root_dir, file)
                arcname = os.path.relpath(file_path, temp_dir)
                zipf.write(file_path, arcname)
    
    shutil.rmtree(temp_dir)

if __name__ == "__main__":
    create_multi_page_odg("template.odg", "guests.ods", "output.odg")